In [60]:
from dotenv import load_dotenv
load_dotenv() # to load openai_key

import os


In [23]:
#from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [24]:
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader

#reading the doc
def read_doc(dir):
    file_loader = PyPDFDirectoryLoader(dir)
    #file_loader = PdfReader(dir)
    documetns = file_loader.load()
    return documetns

documents = read_doc("../document")


In [25]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs = text_splitter.split_documents(documents)

#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
#embeddings = embeddings.generate_embeddings(docs)


In [26]:
from langchain_pinecone import PineconeVectorStore
index_name = "vector-db-1"

In [27]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)


In [28]:
docsearch

In [29]:
#cosine similariry retrive result from vector db

def retrive_result(query,k=3):
    matching_result = docsearch.similarity_search(query=query,k=k)
    return matching_result

In [47]:
#using retrievalqa
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI


llm = OpenAI(temperature=0.6)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 3}),
)



In [48]:
print(qa.run("How much the agriculture target will be increased by how many crore?"))

 The agriculture credit target will be increased to ` 20 lakh crore.


In [44]:
#search answer in vectordb
def find_answer_using_RetrievalQA(query):
    similar_results = retrive_result(query=query)
    print(similar_results)

    result = qa.run(input_documents=similar_results,query=query)
    return result


In [54]:
ans = find_answer_using_RetrievalQA(query="How much the agriculture target will be increased by how many crore?")

[Document(page_content="7 \n \n \n farmers in contributing to the health of fellow citizens by growing these \n‘Shree Anna’.   \n22. Now to make India a global hub for ' Shree Anna' , the Indian Institute \nof Millet Research, Hyderabad  will be supported as the Centre of Excellence \nfor sharing best practices, research and technologies at the international \nlevel.    \nAgriculture Credit  \n23. The agriculture credit target will be increased  \nto ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.", metadata={'page': 10.0, 'source': '..\\document\\budget_speech.pdf'}), Document(page_content='year in a row by 33 per cent to ` 10 lakh crore, which would be 3.3 per cent \nof GDP. This will be almost three times the outlay in 2019-20.   \n45. This substantial increase in recent years is central to the \ngovernment’s efforts to enhance growth potential and job creation, crowd-\nin private investments, and provide a cushion against global headwinds. \nEffective Capital E

In [55]:
ans

' The agriculture credit target will be increased to ` 20 lakh crore.'

In [56]:
#using retrievalqa
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI


llm = OpenAI(temperature=0.6)
qa_1 = load_qa_chain(llm=llm,chain_type="stuff")



In [57]:
#search answer in vectordb
def find_answer(query):
    similar_results = retrive_result(query=query)
    print(similar_results)

    result = qa_1.run(input_documents=similar_results,question=query)
    return result


In [58]:
ans = find_answer("How much the agriculture target will be increased by how many crore?")

[Document(page_content="7 \n \n \n farmers in contributing to the health of fellow citizens by growing these \n‘Shree Anna’.   \n22. Now to make India a global hub for ' Shree Anna' , the Indian Institute \nof Millet Research, Hyderabad  will be supported as the Centre of Excellence \nfor sharing best practices, research and technologies at the international \nlevel.    \nAgriculture Credit  \n23. The agriculture credit target will be increased  \nto ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.", metadata={'page': 10.0, 'source': '..\\document\\budget_speech.pdf'}), Document(page_content='year in a row by 33 per cent to ` 10 lakh crore, which would be 3.3 per cent \nof GDP. This will be almost three times the outlay in 2019-20.   \n45. This substantial increase in recent years is central to the \ngovernment’s efforts to enhance growth potential and job creation, crowd-\nin private investments, and provide a cushion against global headwinds. \nEffective Capital E

In [59]:
ans

' The agriculture credit target will be increased to ` 20 lakh crore.'

In [ ]:
#could have used RetrievalQAWithSourcesChain or load_qa_with_sources_chain